<h1>
<center>CFRM 521, Spring 2020</center>
</h1>

<h1>
<center>[Please insert your name(s) here]</center>
</h1>

<h1>
<center>Homework 4</center>
</h1>



* **Due: Monday 1st June 2020, at 11:59pm**

* You may submitted this homework with one other classmate. If you are submitting in a group of two, only one of you must submit the homework. The other must make a comment on Canvas (where you would submit the homework) mentioning that they are submitting with another person. Also, please include both of your names in the homework submission.


* Late homework are allowed, **but a 10% penalty per day applies.** Example: submitting on Tuesday 00:01am results in 10% penalty, on Wednesday 00:01am results in 20% penalty, etc. Your last submission is considered for calculating the penalty.


*  Please use this Jupyter notebook as a template for your solutions. Your solution must be submitted as one Jupyter notebook. You are allowed to use code from the textbook, textbook website, or lecture notes. However, please go over the code you are using.

# 1. 

## (a)

Build a DNN with five hidden layers of 100 neurons each, He initialization, and the ELU activation function.

**[Add your solution here]**

## (b)

Using Adam optimization and early stopping, try training it on MNIST but only on digits 0 to 4, as we will use transfer learning for digits 5 to 9 in the next exercise. You will need a softmax output layer with five neurons, and as always make sure to save checkpoints at regular intervals and save the final model so you can reuse it later.

**[Add your solution here]**

## (c)

Tune the hyper-parameters using cross-validation and see what precision you can achieve.

**[Add your solution here]**

## (d)

Add Batch Normalization to the best model in part (c). Compare the learning curves: is it converging faster than before? Does it produce a better model?

**[Add your solution here]**

## (e)

Is the model over-fitting the training set? Try adding dropout to every layer and try again. Does it help?

**[Add your solution here]**

# 2.

## (a)

create a new DNN that reuses all the pretrained hidden layers of the best model in Q1-(c). Freeze the layers and replaces the softmax output layer with a new layer.

**[Add your solution here]**

## (b)

Train the DNN of part (a) on digits 5 to 9 from MNIST, using only 100 images per digit (that is, only a fraction of the training set). Time how long it takes. Despite this small number of observation, can you achieve a high validation accuracy?

**[Add your solution here]**

## (c)

Try again reusing just four hidden layers instead of five. That is, unfreeze and train the fifth hidden layer. Can you achieve a higher validation accuracy?

**[Add your solution here]**

## (d)

Now unfreeze the top two hidden layers and continue training. Report the validation accuracy of your model.

**[Add your solution here]**

# 3. Implementing ResNet-34

In this problem, we train a ResNet CNN on the MNIST data set. You do your own research how to build a ResNet model, although it may be a bit time consuming.

Here is one way to create it. First, we define a custom residual residual unit.

In [ ]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

Then, we create ResNet-34 CNN as follows. The only (slightly) complicated step is the creation of the residual unit. See the for-loop below which stacks the residual units.

In [ ]:
model = keras.models.Sequential()
model.add(DefaultConv2D(64, kernel_size=7, strides=2,
                        input_shape=[28, 28, 1]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))

# (a)

How many parameters does your CNN have? How many of them are trainable (i.e. they are subject to back propagation)?

**[Add your solution here]**

## (b) Training on MNIST

Let us load the MNIST data set.

In [ ]:
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]
X_train.shape

Train the ResNet-34 model for at least 2 epochs, using `nadam` algorithm, on the whole training set. What accuracy do you get on the validation and test sets?

**Hint:** One epoch takes around 24min on my machine.

**[Add your solution here]**